In [1]:
import os

import cv2
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
ingredients_path = "../data/nutrition5k_dataset_nosides/processed/ingredients_metadata.csv"
dishes_info_path = "../data/nutrition5k_dataset_nosides/processed/dishes_info.csv"
dish_ids_path = "../data/nutrition5k_dataset_nosides/dish_ids/splits/rgb_train_ids.txt"
img_dir = "../data/nutrition5k_dataset_nosides/imagery/realsense_overhead/"

In [3]:
df = pd.read_csv(ingredients_path)
labels = df["ingredient_id"].unique()
labels


array([508, 122,  26, 524,  94,  23, 189,  54,  29, 328, 291, 520, 161,
       462, 525, 312, 513,  43, 448, 514, 152,  45,   6, 116, 471,  31,
       347, 453,  32, 523, 526, 515,  14, 367, 349,   8, 192, 440, 128,
       437, 335, 433,   7,  21, 244,  36, 148, 518, 112,  27, 510,  13,
       517, 201,  46, 126, 124, 133, 332, 540, 257, 434,  11,  92, 341,
        25,  74, 485, 203,  77,   5,  73, 180, 380, 535, 528,  15, 392,
        10,  72,  38,  30,   1, 121, 521, 516,  28, 475,  37, 251,  47,
        52, 320,  12, 529,  49, 527, 205, 522, 441, 519, 428, 194,  33,
        80, 213, 511,  39, 538, 539,  34, 472,  16, 532, 155, 187, 306,
       140, 473, 153,   4, 377, 275, 179, 534, 457, 162,  35, 507,   2,
       512,  42,  82, 129,  58,  41, 421, 458, 338, 545, 543, 502, 484,
       238, 495, 264,  50, 541,  70, 531,   3,  59, 425, 166, 141,  95,
       498,  60, 402, 294,  40, 105, 357,  81,  67, 337, 319, 415, 262,
       423, 364, 297, 113, 342, 372, 455, 273, 463, 102,   9,  1

In [16]:
class IngredientDataset(Dataset):
    def __init__(self, img_dir: str, ingredients_path: str, dish_info_path: str):
        self.img_dir = img_dir

        self.ingredients_df = pd.read_csv(ingredients_path)
        self.dish_info_df = pd.read_csv(dish_info_path)

    def __len__(self) -> int:
        return len(self.dish_info_df)

    def __getitem__(self, index):
        dish = self.dish_info_df.iloc[index]
        dish_id = dish[0]

        ingredient_ids = self.ingredients_df[self.ingredients_df["dish_id"] == dish_id]["ingredient_id"].values
        label_tensor = torch.FloatTensor(ingredient_ids)
        
        dish_weight = dish[2]
        weight_in_g_tensor = torch.FloatTensor([dish_weight])

        img_path = os.path.join(self.img_dir, dish_id, "rgb.png")
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img_tensor = torch.FloatTensor(img)[None, :, :]

        return img_tensor, label_tensor, weight_in_g_tensor

    def get_num_of_classes(self) -> int:
        return self.label_binarizer.classes_.shape[0]

In [17]:
dataset = IngredientDataset(img_dir, ingredients_path, dishes_info_path)

In [18]:
dataset[0]

/var/folders/6b/g5x_vfn514d15rfps5h6pz640000gp/T/ipykernel_6621/2608603742.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dish_id = dish[0]
/var/folders/6b/g5x_vfn514d15rfps5h6pz640000gp/T/ipykernel_6621/2608603742.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dish_weight = dish[2]


(tensor([[[[114., 119., 122.],
           [116., 121., 124.],
           [116., 122., 122.],
           ...,
           [121., 127., 134.],
           [130., 130., 140.],
           [134., 134., 144.]],
 
          [[114., 119., 122.],
           [116., 121., 124.],
           [116., 122., 122.],
           ...,
           [121., 127., 134.],
           [128., 130., 140.],
           [131., 133., 143.]],
 
          [[112., 120., 122.],
           [114., 122., 124.],
           [116., 121., 124.],
           ...,
           [121., 127., 134.],
           [128., 130., 140.],
           [131., 133., 143.]],
 
          ...,
 
          [[105., 112., 116.],
           [102., 109., 113.],
           [102., 110., 112.],
           ...,
           [129., 127., 133.],
           [132., 129., 138.],
           [136., 133., 142.]],
 
          [[ 95., 103., 110.],
           [100., 108., 115.],
           [102., 109., 113.],
           ...,
           [129., 126., 135.],
           [133., 129.,